In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import json
from collections import Counter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#sys.path.insert(0, '/mnt/d/projects/bassline_transcription')
sys.path.insert(0, '/scratch/users/udemir15/ELEC491/bassline_transcription')

from representation import NN_output_to_MIDI_file, replace_sustain, encode_midi_sequence
from dataset import *
from utilities import *

# Read Metadata

In [2]:
directories = get_directories('../data/directories.json')
_, track_dicts, track_titles = read_metadata(directories, 'TechHouse_total_track_dicts.json')

## Load Analysis Files

In [3]:
with open('../data/metadata/bad_beat_f0_titles.txt', 'r') as infile:
    bad_titles = infile.read().split('\n')
    
# cross validation results
#with open('../data/metadata/removed_tracks.txt', 'r') as infile:
#    removed_titles = infile.read().split('\n')   
#bad_titles = bad_titles + removed_titles
print(len(bad_titles))

4420


# Create dataframes from the Transcriber Outputs

In [ ]:
M=8

representation_path = os.path.join('..','data','datasets')
df_minor, df_major = create_dataframes(track_dicts, bad_titles, M, directories,  MAX_NOTE=60, MIN_NOTE=24)

m_counter = count_keys(df_minor, track_dicts)
M_counter = count_keys(df_major, track_dicts)

key_pie_charts(m_counter, M_counter)

In [ ]:
df = df_minor.copy()

## Fix to a Single Octave

In [ ]:
titles = df.iloc[:,0].tolist()
X = df.iloc[:,1:].to_numpy()  
np.unique(X)

MAX_NOTE = 24
X = replace_sustain(X, MAX_NOTE+1)
np.unique(X)

In [ ]:
MAX_NOTE = 37
X = replace_sustain(X, MAX_NOTE+1)

print(np.unique(X))
X[X<13] += 12
print(np.unique(X))
X[X>=13+12+12] -= 24
print(np.unique(X))
X[X>=13+12] -= 12
print(np.unique(X))
X -= 12
print(np.unique(X))

# put sustain
X = np.stack([encode_midi_sequence(code, sustain_code=13) for code in X], axis=0)

In [ ]:
df_minor_new = make_dataframe(X, titles)
df_minor_new.head()

## Groove Dataset

In [ ]:
np.unique(X)

In [ ]:
X[X!=0] = 1

df_minor_new = make_dataframe(X, titles)

In [ ]:
dataset_name = 'TechHouse_bassline_representations_4521'

representation_path = os.path.join('..','data','datasets', 'groove') #'[28, 51]'
os.makedirs(representation_path, exist_ok=True)
min_title = dataset_name+'_min_M{}.csv'.format(M)
min_dir = os.path.join(representation_path, min_title)
df_minor_new.to_csv(min_dir, index=False, header=False)

## Exporting

M=8
dataset_name = 'TechHouse_bassline_representations_4020'
representation_path = os.path.join('..','data','datasets', '[28, 51]', 'no_sus')

os.makedirs(representation_path, exist_ok=True)
min_title = dataset_name+'_min_M{}.csv'.format(M)
min_dir = os.path.join(representation_path, min_title)
df.to_csv(min_dir, index=False, header=False)

In [ ]:
dataset_name = 'TechHouse_bassline_representations_4020'

representation_path = os.path.join('..','data','datasets', '[28, 51]', 'sus') #'[28, 51]'
os.makedirs(representation_path, exist_ok=True)
min_title = dataset_name+'_min_M{}.csv'.format(M)
min_dir = os.path.join(representation_path, min_title)
df_minor.to_csv(min_dir, index=False, header=False)

dataset_name = 'TechHouse_bassline_representations'

representation_path = os.path.join('..','data','datasets','[28, 51]')

for M in [1, 2, 4, 8]:
    df_minor, df_major = create_dataframes(track_dicts, bad_titles, M, directories)

    min_title = dataset_name+'_min_M{}.csv'.format(M)
    min_dir = os.path.join(representation_path, min_title)
    df_minor.to_csv(min_dir, index=False, header=False)

    maj_title = dataset_name+'_maj_M{}.csv'.format(M)
    maj_dir = os.path.join(representation_path, maj_title)
    df_major.to_csv(maj_dir, index=False, header=False)

# Merging Track Dicts

In [ ]:
directories = get_directories('../data/directories.json')

track_dicts_name = 'traxsource_crawl_0_2500_track_dicts.json'
_, track_dicts0, track_titles0 = read_metadata(directories, track_dicts_name)

track_dicts_name = 'traxsource_crawl_2500_5000_track_dicts.json'
_, track_dicts1, track_titles1 = read_metadata(directories, track_dicts_name)

track_dicts_name = 'traxsource_crawl_5000_7500_track_dicts.json'
_, track_dicts2, track_titles2 = read_metadata(directories, track_dicts_name)

track_dicts_name = 'traxsource_crawl_7500_10000_track_dicts.json'
_, track_dicts3, track_titles3 = read_metadata(directories, track_dicts_name)

track_dicts_name = 'traxsource_crawl_10000_10898_track_dicts.json'
_, track_dicts4, track_titles4 = read_metadata(directories, track_dicts_name)

track_dicts_name = 'TechHouse_track_dicts.json'
_, track_dicts5, track_titles3 = read_metadata(directories, track_dicts_name)

track_dicts = merge_track_dicts(track_dicts0, track_dicts1, track_dicts2, track_dicts3, track_dicts4, track_dicts5)
track_titles = list(track_dicts.keys())

with open('../data/metadata/TechHouse_total_track_dicts.json', 'w') as outfile:
    json.dump(track_dicts, outfile, indent=4)

## provide non-transposed midi sequences (doesn't funtion now) 

note_counter, note_counter_T = count_notes(track_dicts, directories, M)

plot_note_occurances(note_counter, note_counter_T, M)